<a href="https://colab.research.google.com/github/farhanhubble/google-colab/blob/master/advanced_lane_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [11]:
!ls /content/drive/My\ Drive/data_driver_view | head -10

00002207.png
00002208.png
00002209.png
00002210.png
00002211.png
00002212.png
00002213.png
00002214.png
00002215.png
00002216.png


In [0]:
def plot_img_grid(images, nb_cols):
    nb_images = len(images)
    nb_rows = int(np.ceil(nb_images/nb_cols))
    
    for i in range(nb_rows):
        plt.figure(figsize=(32, 4))
        for j in range(nb_cols):
            plt.subplot(1, nb_cols, j+1)
            image = images[i*nb_cols + j]
            nb_channels = 1 if len(image.shape) == 2 else image.shape[-1]
            plt.imshow(image, cmap='gray' if nb_channels == 1 else None)
            plt.xticks([])
            plt.yticks([])
        plt.show()

In [0]:
road_data = []

In [15]:
from glob import glob
filenames = glob('/content/drive/My Drive/data_driver_view/*.png')
filenames[:10]

['/content/drive/My Drive/data_driver_view/00002212.png',
 '/content/drive/My Drive/data_driver_view/00002218.png',
 '/content/drive/My Drive/data_driver_view/00002208.png',
 '/content/drive/My Drive/data_driver_view/00002207.png',
 '/content/drive/My Drive/data_driver_view/00002214.png',
 '/content/drive/My Drive/data_driver_view/00002209.png',
 '/content/drive/My Drive/data_driver_view/00002213.png',
 '/content/drive/My Drive/data_driver_view/00002216.png',
 '/content/drive/My Drive/data_driver_view/00002215.png',
 '/content/drive/My Drive/data_driver_view/00002210.png']

In [0]:
for name in filenames:
  img_bgr = cv2.imread(name)
  img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
  road_data.append(img_rgb)